In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.optimize as opt
from functools import partial
import pandas as pd

# Structural Econometrics in Labor and IO.

# Problem Set: Dynamic Discrete Choice Estimation.

  
## Description

Superintendent Harry Zurcher manages the bus fleet at the Madison Metropolitan Bus Com- pany. Notably, at every point in time t, he decides whether to replace an old bus engine with a new one or to keep operating the old one.
We are interested in estimating the cost of engine maintenance and replacement, using observed data on:

1. mileage $x_t$ at period $t$,
2. observed replacement decision $i_t$ at period $t$, and
3. mileage $x_t+1$ next period.

#### The data generating process is as follows:

- $x_t$ can take on 11 values, $x_t \in \{0, 1, 2, . . . , 10\} (i.e. mileage discretized into 11 categories).

- At any time period,
\begin{equation*}
x_{t+1} =
\begin{cases}
    \min\{x_t+1, \ 10\},& \text{ with probability } \lambda, \\
    x_t,& \text{ with probability } 1-\lambda.
\end{cases}
\end{equation*}
Note that $\lambda$ does not depend on the replacement decision $i_t$.
- The agent’s replacement decision is made at the beginning of each period and is effective immediately (i.e. if $i_t = 1$, the agent uses machine with mileage $0$ and the next period state $x_t+1$ is $1$ with probability $\lambda$ and $0$ with probability $1 − \lambda$).
- The per-period maintenance cost for a bus with mileage $x$ is $C(x, \theta) = \theta_1 x + \theta_2 x^2$.
- The cost of replacement is $\theta_3$.
- The per-period utility function is given by
\begin{equation*}
u(x_t,i_t,\epsilon_{1t},\epsilon_{2t};\theta) =
\begin{cases}
    −\theta_1 x_t − \theta_2 x^2_t + \epsilon_{0t},& \text{ if } i_t = 0,\\
    −\theta_3 + \epsilon_{1t},& \text{ if } i_t = 1,
\end{cases}
\end{equation*}
where $(\epsilon_{0t}, \epsilon_{1t})$ ~ iid type 1 extreme value.
- Assume that the decision-maker's discount factor is known, $\beta = 0.95$.
- The number of observations is $N = 1000$.
- The true parameters are: $\theta_1 = 0.5$, $\theta_2 = 0.04$, $\theta_3 = 2$, and $\lambda = 0.82$.
- Hint: All busses are assumed to be operated and maintained independently.


In [2]:
# Import custom functions from python files. For example, I define and use these functions in separate files:
# from generate_data import simulate_data
# from dynamicprogramming import bellman
# from estimation import rust_llf

In [3]:
seed = 100
np.random.seed(seed)

Set parameters.

In [4]:
# Setting the parameters
theta_1 = 0.5
theta_2 = 0.04
theta_3 = 2
_lambda = 0.82
beta = 0.95


In [5]:
#Setting number of observations
N = 1000

#Defining mileage variable as states
mileage = np.arange(11)

#Defining the decision rule where 0=keep and 1=replace
d_rule = [0,1]

#Defining the utility function and maintenance cost functions
def maintenance_cost(x):
    return theta_1 * x + theta_2 * x**2

def utility(x, i, eps0, eps1):
    if i == 0:
        return -maintenance_cost(x) + eps0
    else:
        return -theta_3 + eps1

#Defining the transition probabilities
def transition_probs(x, decision, lambda_):
    probs = np.zeros(11)
    if decision == 0:  #we keep the engine
        next_x = min(x + 1, 10)
        probs[next_x] = lambda_
        probs[x] += 1 - lambda_
    elif decision == 1: #we replace the engine
        probs[1] = lambda_
        probs[0] = 1 - lambda_
    return probs

## Problem 1. Simulate data.

Solve dynamic programming problem to obtain replacement probabilities for each mileage state.

In [6]:
from scipy.special import logsumexp

#Defining mileage range globally
mileage = np.arange(11)

def solve_dp(theta_1, theta_2, theta_3, _lambda, beta):
    """
    Solves the dynamic programming problem for optimal replacement.

    Parameters:
        theta_1 (float): Linear maintenance cost parameter
        theta_2 (float): Quadratic maintenance cost parameter
        theta_3 (float): Replacement cost
        _lambda (float): Probability of transitioning to higher mileage
        beta (float): Discount factor

    Returns:
        tuple: (prob_replace, prob_keep, V)
            prob_replace (ndarray): Probability of replacing at each mileage
            prob_keep (ndarray): Probability of keeping at each mileage
            V (ndarray): Value function at each mileage
    """
    n_states = len(mileage)
    V = np.zeros(n_states)
    tol = 1e-8
    max_iter = 10000

    maint_costs = theta_1 * mileage + theta_2 * mileage**2

    for _ in range(max_iter):
        V_old = V.copy()
        EV_keep = np.zeros(n_states) #Value of keeping
        EV_replace = np.zeros(n_states) #Value of replacing

        for x in range(n_states):
            next_x1 = min(x + 1, n_states - 1)
            EV_keep[x] = _lambda * V[next_x1] + (1 - _lambda) * V[x]
        EV_replace[:] = _lambda * V[1] + (1 - _lambda) * V[0]

        v_keep = -maint_costs + beta * EV_keep
        v_replace = -theta_3 + beta * EV_replace
        V = logsumexp(np.vstack([v_keep, v_replace]), axis=0)

        if np.max(np.abs(V - V_old)) < tol:
            break

    prob_replace = np.exp(v_replace - V)
    prob_keep = np.exp(v_keep - V)
    return prob_replace, prob_keep, V


#Solving Dynamic Programming
prob_replace, prob_keep, V = solve_dp(theta_1, theta_2, theta_3, _lambda, beta)

#Printing the results
for x in range(len(mileage)):
    print(f"Mileage {x}: P(replace) = {prob_replace[x]:.4f}, "
          f"P(keep) = {prob_keep[x]:.4f}, Value = {V[x]:.4f}")

Mileage 0: P(replace) = 0.1192, P(keep) = 0.8808, Value = -11.7361
Mileage 1: P(replace) = 0.2980, P(keep) = 0.7020, Value = -12.6523
Mileage 2: P(replace) = 0.5285, P(keep) = 0.4715, Value = -13.2254
Mileage 3: P(replace) = 0.7315, P(keep) = 0.2685, Value = -13.5504
Mileage 4: P(replace) = 0.8671, P(keep) = 0.1329, Value = -13.7204
Mileage 5: P(replace) = 0.9415, P(keep) = 0.0585, Value = -13.8027
Mileage 6: P(replace) = 0.9767, P(keep) = 0.0233, Value = -13.8395
Mileage 7: P(replace) = 0.9915, P(keep) = 0.0085, Value = -13.8545
Mileage 8: P(replace) = 0.9972, P(keep) = 0.0028, Value = -13.8602
Mileage 9: P(replace) = 0.9991, P(keep) = 0.0009, Value = -13.8622
Mileage 10: P(replace) = 0.9998, P(keep) = 0.0002, Value = -13.8628


Simulate data.

In [7]:
#Getting choice probabilities based on true θ
prob_replace, _, _ = solve_dp(theta_1, theta_2, theta_3, _lambda, beta)

#Initialize arrays
xt = np.zeros(N, dtype=int)
it = np.zeros(N, dtype=int)
xt1 = np.zeros(N, dtype=int)

#Initial mileage state
x = np.random.randint(0, 11)

#Simulating using logit choice probabilities
for t in range(N):
    xt[t] = x

    #Sample replacement decision from probability
    i = int(np.random.rand() < prob_replace[x])
    it[t] = i

    #Simulating next state based on action
    if i == 1:
        #Replaced: new engine, so go to 0 or 1
        x_next = 1 if np.random.rand() < _lambda else 0
    else:
        #Kept: either stays or increments mileage
        x_next = min(x + 1, 10) if np.random.rand() < _lambda else x

    xt1[t] = x_next
    x = x_next

#Store results in DataFrame
sim_data = pd.DataFrame({'xt': xt, 'it': it, 'xt1': xt1})
sim_data


,xt,it,xt1
0,8,1,1
1,1,0,2
2,2,1,1
3,1,1,1
4,1,0,2
...,...,...,...
995,1,0,2
996,2,0,3
997,3,1,0
998,0,0,1


## Problem 2. Estimate the model using the simulated data.

Estimate and report transition probability λ in a first step. This does not require any nonlinear optimization.

In [8]:
numerator = ((sim_data['it'] == 0) & (sim_data['xt1'] == sim_data['xt'].clip(upper=9) + 1)).sum()
denominator = (sim_data['it'] == 0).sum()

lambda_hat = numerator / denominator
print(f"Estimated λ: {lambda_hat:.4f}")

Estimated λ: 0.8101


Set parameters for maximum likelihood estimation.

In [9]:
theta_init = np.array([0.1, 0.05, 1])  # initial guess for theta_1, theta_2, theta_3
beta = 0.95

Report the parameters of the model estimated using MLE with a nested fixed point algorithm:

1. Guess initial parameter values.

2. Solve dynamic programming problem.
3. Calculate the probability of replacement at each state.
4. Use model predictions (the probability calculated in the previous step) and data to compute the log-likelihood function.
5. Search over parameter values by repeating steps 2 − 4.

The objective function can be minimized using *scipy.optimize.minimize*.

In [10]:
def solve_value_functions(theta, _lambda, beta=0.95, tol=1e-8, max_iter=1000):
    theta_1, theta_2, theta_3 = theta
    n_states = 11
    mileage = np.arange(n_states)

    # Initialize value functions
    V0 = np.zeros(n_states)  # Value of keeping
    V1 = np.zeros(n_states)  # Value of replacing

    # Build transition matrices
    P_keep = np.zeros((n_states, n_states))
    P_replace = np.zeros((n_states, n_states))

    for x in range(n_states):
        # Transition for keeping
        next_x = min(x + 1, n_states - 1)
        P_keep[x, next_x] = _lambda
        P_keep[x, x] = 1 - _lambda

        # Transition for replacing
        P_replace[x, 1] = _lambda
        P_replace[x, 0] = 1 - _lambda

    for _ in range(max_iter):
        V0_old, V1_old = V0.copy(), V1.copy()

        # Compute logsumexp of current value functions
        logsum = logsumexp(np.vstack([V0_old, V1_old]), axis=0)

        # Expected continuation value
        E_logsum_keep = P_keep @ logsum
        E_logsum_replace = P_replace[0] @ logsum  # same for all since replace resets state

        # Flow utilities
        u_keep = -(theta_1 * mileage + theta_2 * mileage**2)
        u_replace = -theta_3

        # Bellman updates
        V0 = u_keep + beta * E_logsum_keep
        V1 = np.full(n_states, u_replace + beta * E_logsum_replace)

        # Convergence check
        if np.max(np.abs(V0 - V0_old)) < tol and np.max(np.abs(V1 - V1_old)) < tol:
            break

    return V0, V1

In [11]:
def compute_choice_probabilities(x, V0, V1):
    denom = np.exp(V0[x]) + np.exp(V1[x])
    return np.exp(V1[x]) / denom


In [12]:
def log_likelihood(theta, data, _lambda, beta=0.95):
    V0, V1 = solve_value_functions(theta, _lambda, beta)
    loglike = 0.0
    for index, row in data.iterrows():
        x = row['xt']
        i = row['it']
        p = compute_choice_probabilities(x, V0, V1)
        if i == 1:
            loglike += np.log(p + 1e-10)
        else:
            loglike += np.log(1 - p + 1e-10)
    return -loglike  # For minimization


In [13]:
from scipy.optimize import minimize
import numpy as np

def estimate_theta(data, lambda_hat):
    # Improved initial guess (closer to true θ values)
    theta_init = np.array([0.5, 0.04, 2.0])

    result = minimize(
        fun=log_likelihood,
        x0=theta_init,
        args=(data, lambda_hat),
        method='L-BFGS-B',
        options={'disp': True, 'maxiter': 1000}
    )

    return result.x, result.fun

Report the estimated parameters and the probability of replacement for every state: $P (1|x_t, \theta)$.

In [14]:
theta_hat, neg_loglike = estimate_theta(sim_data, lambda_hat)
print("Estimated θ:", theta_hat)
print("True θ: [0.5, 0.04, 2.0]")
print("Negative log-likelihood:", neg_loglike)

Estimated θ: [0.55400859 0.06800862 2.16954639]
True θ: [0.5, 0.04, 2.0]
Negative log-likelihood: 584.7614351977107


In [15]:
# Solve value function at estimated theta (Rust-style)
V0_hat, V1_hat = solve_value_functions(theta_hat, lambda_hat)

# Print predicted replacement probabilities
print("Predicted Replacement Probabilities:")
for x in range(11):
    p = compute_choice_probabilities(x, V0_hat, V1_hat)
    print(f"Mileage {x}: P(replace) = {p:.4f}")


Predicted Replacement Probabilities:
Mileage 0: P(replace) = 0.1025
Mileage 1: P(replace) = 0.2977
Mileage 2: P(replace) = 0.5679
Mileage 3: P(replace) = 0.7928
Mileage 4: P(replace) = 0.9200
Mileage 5: P(replace) = 0.9743
Mileage 6: P(replace) = 0.9929
Mileage 7: P(replace) = 0.9983
Mileage 8: P(replace) = 0.9997
Mileage 9: P(replace) = 0.9998
Mileage 10: P(replace) = 0.1809
